In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from keras.layers import Dense, Flatten
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import keras
from keras.datasets import cifar10

from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout

from keras.regularizers import l1, l2

from tensorflow.keras.layers import LeakyReLU

In [2]:
# define num_class
num_classes = 10

# load dataset keras will download cifar-10 datset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()  

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# Converting class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [4]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Models

### Relu activation function

In [5]:
# Designing the layer architecture of the DNN model
LAYERS = [
          tf.keras.layers.Flatten(input_shape=[32, 32,3], name="inputLayer"),
          tf.keras.layers.Dense(300, activation="relu", name="hiddenLayer1"),
          tf.keras.layers.Dense(200, activation="relu", name="hiddenLayer2"),
          tf.keras.layers.Dense(100, activation="relu", name="hiddenLayer3"),
          tf.keras.layers.Dense(10, activation="softmax", name="outputLayer"),
]

In [6]:
model_relu = tf.keras.models.Sequential(LAYERS)

2023-05-12 14:03:09.570973: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-12 14:03:09.571751: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro


In [7]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model_relu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [8]:
history_relu = model_relu.fit(x_train,y_train, 
              epochs=50,batch_size = 128,
              validation_data=(x_test, y_test))

Epoch 1/50


2023-05-12 14:04:12.096387: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-12 14:04:12.411327: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 1.8799 - accuracy: 0.3216 - precision: 0.6066 - recall: 0.0674

2023-05-12 14:04:19.830357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 9s 16ms/step - loss: 1.8799 - accuracy: 0.3216 - precision: 0.6066 - recall: 0.0674 - val_loss: 1.7541 - val_accuracy: 0.3739 - val_precision: 0.6326 - val_recall: 0.1035
Epoch 2/50
391/391 [==============================] - 6s 15ms/step - loss: 1.6788 - accuracy: 0.4021 - precision: 0.6334 - recall: 0.1469 - val_loss: 1.6515 - val_accuracy: 0.4058 - val_precision: 0.6131 - val_recall: 0.1748
Epoch 3/50
391/391 [==============================] - 6s 15ms/step - loss: 1.5961 - accuracy: 0.4316 - precision: 0.6485 - recall: 0.1900 - val_loss: 1.5659 - val_accuracy: 0.4464 - val_precision: 0.6868 - val_recall: 0.1949
Epoch 4/50
391/391 [==============================] - 6s 15ms/step - loss: 1.5336 - accuracy: 0.4518 - precision: 0.6654 - recall: 0.2236 - val_loss: 1.5333 - val_accuracy: 0.4583 - val_precision: 0.6665 - val_recall: 0.2384
Epoch 5/50
391/391 [==============================] - 6s 15ms/step - loss: 1.4919 - accuracy: 0.4668 - precisio

391/391 [==============================] - 6s 15ms/step - loss: 0.9118 - accuracy: 0.6716 - precision: 0.7877 - recall: 0.5595 - val_loss: 1.5166 - val_accuracy: 0.5150 - val_precision: 0.6131 - val_recall: 0.4125
Epoch 38/50
391/391 [==============================] - 6s 15ms/step - loss: 0.9062 - accuracy: 0.6735 - precision: 0.7863 - recall: 0.5621 - val_loss: 1.5226 - val_accuracy: 0.5198 - val_precision: 0.6111 - val_recall: 0.4309
Epoch 39/50
391/391 [==============================] - 6s 15ms/step - loss: 0.8911 - accuracy: 0.6783 - precision: 0.7900 - recall: 0.5708 - val_loss: 1.5722 - val_accuracy: 0.5105 - val_precision: 0.6015 - val_recall: 0.4239
Epoch 40/50
391/391 [==============================] - 6s 15ms/step - loss: 0.8851 - accuracy: 0.6815 - precision: 0.7923 - recall: 0.5743 - val_loss: 1.5862 - val_accuracy: 0.5131 - val_precision: 0.6028 - val_recall: 0.4300
Epoch 41/50
391/391 [==============================] - 6s 15ms/step - loss: 0.8672 - accuracy: 0.6880 - prec

In [9]:
model_relu.evaluate(x_test, y_test)

313/313 [==============================] - 3s 11ms/step - loss: 1.7091 - accuracy: 0.5138 - precision: 0.5901 - recall: 0.4466


[1.7091299295425415,
 0.5138000249862671,
 0.5901162624359131,
 0.4466000199317932]

Training accuracy: 71%
Testing accuracy: 51.38%

### Leaky-relu activation function

In [10]:
# Designing the layer architecture of the DNN model
LAYERS = [
          tf.keras.layers.Flatten(input_shape=[32, 32,3], name="inputLayer"),
          tf.keras.layers.Dense(300, activation=tf.keras.layers.LeakyReLU(alpha=0.01), name="hiddenLayer1"),
          tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.01), name="hiddenLayer2"),
          tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.01), name="hiddenLayer3"),
          tf.keras.layers.Dense(10, activation="softmax", name="outputLayer"),
]

In [11]:
model_Leakyrelu = tf.keras.models.Sequential(LAYERS)

In [12]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model_Leakyrelu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [13]:
history_Leakyrelu = model_Leakyrelu.fit(x_train,y_train, 
              epochs=50,batch_size = 128,
              validation_data=(x_test, y_test))

Epoch 1/50


2023-05-12 14:14:02.166733: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 1.9010 - accuracy: 0.3108 - precision: 0.5918 - recall: 0.0556

2023-05-12 14:14:07.766725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 7s 16ms/step - loss: 1.9010 - accuracy: 0.3108 - precision: 0.5918 - recall: 0.0556 - val_loss: 1.7439 - val_accuracy: 0.3703 - val_precision: 0.6160 - val_recall: 0.1110
Epoch 2/50
391/391 [==============================] - 6s 15ms/step - loss: 1.6915 - accuracy: 0.3933 - precision: 0.6278 - recall: 0.1404 - val_loss: 1.6419 - val_accuracy: 0.4105 - val_precision: 0.6237 - val_recall: 0.1843
Epoch 3/50
391/391 [==============================] - 6s 15ms/step - loss: 1.5989 - accuracy: 0.4275 - precision: 0.6560 - recall: 0.1891 - val_loss: 1.5540 - val_accuracy: 0.4517 - val_precision: 0.6997 - val_recall: 0.1857
Epoch 4/50
391/391 [==============================] - 6s 15ms/step - loss: 1.5475 - accuracy: 0.4476 - precision: 0.6594 - recall: 0.2141 - val_loss: 1.5633 - val_accuracy: 0.4451 - val_precision: 0.6311 - val_recall: 0.2246
Epoch 5/50
391/391 [==============================] - 6s 15ms/step - loss: 1.4991 - accuracy: 0.4640 - precisio

391/391 [==============================] - 6s 15ms/step - loss: 0.8692 - accuracy: 0.6890 - precision: 0.7978 - recall: 0.5844 - val_loss: 1.6224 - val_accuracy: 0.5098 - val_precision: 0.5986 - val_recall: 0.4331
Epoch 38/50
391/391 [==============================] - 6s 15ms/step - loss: 0.8603 - accuracy: 0.6894 - precision: 0.7975 - recall: 0.5885 - val_loss: 1.6084 - val_accuracy: 0.5144 - val_precision: 0.6056 - val_recall: 0.4291
Epoch 39/50
391/391 [==============================] - 6s 15ms/step - loss: 0.8412 - accuracy: 0.6982 - precision: 0.8017 - recall: 0.5984 - val_loss: 1.6382 - val_accuracy: 0.5087 - val_precision: 0.5967 - val_recall: 0.4272
Epoch 40/50
391/391 [==============================] - 6s 15ms/step - loss: 0.8266 - accuracy: 0.7038 - precision: 0.8044 - recall: 0.6065 - val_loss: 1.6333 - val_accuracy: 0.5151 - val_precision: 0.5969 - val_recall: 0.4295
Epoch 41/50
391/391 [==============================] - 6s 15ms/step - loss: 0.8127 - accuracy: 0.7086 - prec

In [14]:
model_Leakyrelu.evaluate(x_test, y_test)

313/313 [==============================] - 3s 11ms/step - loss: 1.8305 - accuracy: 0.5031 - precision: 0.5670 - recall: 0.4364


[1.8305155038833618,
 0.5031000375747681,
 0.5670478343963623,
 0.43640002608299255]

Training accuracy: 74.25%
Testing accuracy: 50.3%

### Randomized Leaky Relu

In [23]:
# Custom layer for randomized Leaky ReLU activation
class RandomizedLeakyReLU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(RandomizedLeakyReLU, self).__init__(**kwargs)

    def call(self, inputs):
        alpha = tf.random.uniform(shape=tf.shape(inputs), minval=0.01, maxval=0.2)
        return tf.maximum(alpha * inputs, inputs)

# Modify the layers to use randomized Leaky ReLU activation
LAYERS = [
    tf.keras.layers.Flatten(input_shape=[32, 32, 3], name="inputLayer"),
    tf.keras.layers.Dense(300, name="hiddenLayer1"),
    RandomizedLeakyReLU(name="hiddenLayer1_activation"),
    tf.keras.layers.Dense(200, name="hiddenLayer2"),
    RandomizedLeakyReLU(name="hiddenLayer2_activation"),
    tf.keras.layers.Dense(100, name="hiddenLayer3"),
    RandomizedLeakyReLU(name="hiddenLayer3_activation"),
    tf.keras.layers.Dense(10, activation="softmax", name="outputLayer")
]

In [24]:
model_RandomizedLeakyrelu = tf.keras.models.Sequential(LAYERS)

In [25]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model_RandomizedLeakyrelu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [26]:
history_RandomizedLeakyrelu = model_RandomizedLeakyrelu.fit(x_train,y_train, 
              epochs=50,batch_size = 128,
              validation_data=(x_test, y_test))

Epoch 1/50


2023-05-12 14:32:58.933362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 1.9105 - accuracy: 0.3092 - precision: 0.5714 - recall: 0.0641

2023-05-12 14:33:05.244197: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 8s 18ms/step - loss: 1.9105 - accuracy: 0.3092 - precision: 0.5714 - recall: 0.0641 - val_loss: 1.7491 - val_accuracy: 0.3775 - val_precision: 0.6091 - val_recall: 0.1259
Epoch 2/50
391/391 [==============================] - 7s 17ms/step - loss: 1.7035 - accuracy: 0.3892 - precision: 0.6171 - recall: 0.1420 - val_loss: 1.6783 - val_accuracy: 0.4032 - val_precision: 0.6352 - val_recall: 0.1576
Epoch 3/50
391/391 [==============================] - 7s 17ms/step - loss: 1.6212 - accuracy: 0.4233 - precision: 0.6439 - recall: 0.1861 - val_loss: 1.5678 - val_accuracy: 0.4412 - val_precision: 0.6748 - val_recall: 0.1998
Epoch 4/50
391/391 [==============================] - 7s 18ms/step - loss: 1.5643 - accuracy: 0.4427 - precision: 0.6523 - recall: 0.2138 - val_loss: 1.5940 - val_accuracy: 0.4385 - val_precision: 0.6326 - val_recall: 0.2197
Epoch 5/50
391/391 [==============================] - 7s 17ms/step - loss: 1.5130 - accuracy: 0.4613 - precisio

391/391 [==============================] - 7s 17ms/step - loss: 0.8639 - accuracy: 0.6888 - precision: 0.7922 - recall: 0.5901 - val_loss: 1.4739 - val_accuracy: 0.5357 - val_precision: 0.6225 - val_recall: 0.4506
Epoch 38/50
391/391 [==============================] - 7s 18ms/step - loss: 0.8429 - accuracy: 0.6962 - precision: 0.7970 - recall: 0.6029 - val_loss: 1.5085 - val_accuracy: 0.5272 - val_precision: 0.6158 - val_recall: 0.4478
Epoch 39/50
391/391 [==============================] - 7s 17ms/step - loss: 0.8343 - accuracy: 0.7002 - precision: 0.7966 - recall: 0.6053 - val_loss: 1.5219 - val_accuracy: 0.5264 - val_precision: 0.6112 - val_recall: 0.4487
Epoch 40/50
391/391 [==============================] - 7s 17ms/step - loss: 0.8254 - accuracy: 0.7026 - precision: 0.7986 - recall: 0.6107 - val_loss: 1.4953 - val_accuracy: 0.5463 - val_precision: 0.6227 - val_recall: 0.4645
Epoch 41/50
391/391 [==============================] - 7s 18ms/step - loss: 0.8052 - accuracy: 0.7092 - prec

In [27]:
model_RandomizedLeakyrelu.evaluate(x_test, y_test)

313/313 [==============================] - 4s 12ms/step - loss: 1.6790 - accuracy: 0.5426 - precision: 0.5958 - recall: 0.4807


[1.6789637804031372,
 0.5426000356674194,
 0.5958106517791748,
 0.4807000160217285]

Training accuracy: 74%
Testing Accuracy: 54.26%

### Parametric Leaky Relu

In [29]:

# Modify the layers to use Parametric Leaky ReLU activation
LAYERS = [
    tf.keras.layers.Flatten(input_shape=[32, 32, 3], name="inputLayer"),
    tf.keras.layers.Dense(300, name="hiddenLayer1"),
    tf.keras.layers.PReLU(name="hiddenLayer1_activation"),
    tf.keras.layers.Dense(200, name="hiddenLayer2"),
    tf.keras.layers.PReLU(name="hiddenLayer2_activation"),
    tf.keras.layers.Dense(100, name="hiddenLayer3"),
    tf.keras.layers.PReLU(name="hiddenLayer3_activation"),
    tf.keras.layers.Dense(10, activation="softmax", name="outputLayer")
]

In [30]:
model_ParaLeakyrelu = tf.keras.models.Sequential(LAYERS)

In [31]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model_ParaLeakyrelu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [32]:
history_ParaLeakyrelu = model_ParaLeakyrelu.fit(x_train,y_train, 
              epochs=50,batch_size = 128,
              validation_data=(x_test, y_test))

Epoch 1/50


2023-05-12 14:41:24.969942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 1.8927 - accuracy: 0.3160 - precision: 0.5860 - recall: 0.0594

2023-05-12 14:41:31.530590: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 8s 19ms/step - loss: 1.8927 - accuracy: 0.3160 - precision: 0.5860 - recall: 0.0594 - val_loss: 1.7395 - val_accuracy: 0.3740 - val_precision: 0.6286 - val_recall: 0.1063
Epoch 2/50
391/391 [==============================] - 7s 17ms/step - loss: 1.6822 - accuracy: 0.3975 - precision: 0.6352 - recall: 0.1467 - val_loss: 1.6315 - val_accuracy: 0.4114 - val_precision: 0.6470 - val_recall: 0.1851
Epoch 3/50
391/391 [==============================] - 7s 17ms/step - loss: 1.5830 - accuracy: 0.4347 - precision: 0.6522 - recall: 0.1970 - val_loss: 1.5649 - val_accuracy: 0.4445 - val_precision: 0.6469 - val_recall: 0.2383
Epoch 4/50
391/391 [==============================] - 7s 17ms/step - loss: 1.5331 - accuracy: 0.4512 - precision: 0.6592 - recall: 0.2247 - val_loss: 1.5736 - val_accuracy: 0.4365 - val_precision: 0.6336 - val_recall: 0.2430
Epoch 5/50
391/391 [==============================] - 7s 17ms/step - loss: 1.4801 - accuracy: 0.4731 - precisio

391/391 [==============================] - 7s 17ms/step - loss: 0.8401 - accuracy: 0.6992 - precision: 0.7993 - recall: 0.6023 - val_loss: 1.5735 - val_accuracy: 0.5153 - val_precision: 0.5998 - val_recall: 0.4318
Epoch 38/50
391/391 [==============================] - 7s 17ms/step - loss: 0.8203 - accuracy: 0.7038 - precision: 0.8032 - recall: 0.6126 - val_loss: 1.6756 - val_accuracy: 0.5102 - val_precision: 0.5859 - val_recall: 0.4356
Epoch 39/50
391/391 [==============================] - 7s 17ms/step - loss: 0.8101 - accuracy: 0.7082 - precision: 0.8069 - recall: 0.6185 - val_loss: 1.6403 - val_accuracy: 0.5080 - val_precision: 0.5924 - val_recall: 0.4305
Epoch 40/50
391/391 [==============================] - 7s 17ms/step - loss: 0.7955 - accuracy: 0.7141 - precision: 0.8102 - recall: 0.6266 - val_loss: 1.6790 - val_accuracy: 0.5113 - val_precision: 0.5807 - val_recall: 0.4409
Epoch 41/50
391/391 [==============================] - 7s 17ms/step - loss: 0.7763 - accuracy: 0.7213 - prec

In [33]:
model_ParaLeakyrelu.evaluate(x_test, y_test)

313/313 [==============================] - 4s 12ms/step - loss: 1.8263 - accuracy: 0.5118 - precision: 0.5767 - recall: 0.4486


[1.826294183731079,
 0.5118000507354736,
 0.5766807794570923,
 0.44860002398490906]

training accuracy: 75.88%
testing accuracy: 51.1%

### Exponential Linear Unit

In [44]:
# Modify the layers to use ELU activation
LAYERS = [
    tf.keras.layers.Flatten(input_shape=[32, 32, 3], name="inputLayer"),
    tf.keras.layers.Dense(300, activation="elu", name="hiddenLayer1"),
    tf.keras.layers.Dense(200, activation="elu", name="hiddenLayer2"),
    tf.keras.layers.Dense(100, activation="elu", name="hiddenLayer3"),
    tf.keras.layers.Dense(10, activation="softmax", name="outputLayer")
]

In [45]:
model_elu = tf.keras.models.Sequential(LAYERS)

In [46]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model_elu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [47]:
history_elu = model_elu.fit(x_train,y_train, 
              epochs=50,batch_size = 128,
              validation_data=(x_test, y_test))

Epoch 1/50


2023-05-12 15:07:51.590594: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


207/391 [==============>...............] - ETA: 3s - loss: 2.0367 - accuracy: 0.2800 - precision: 0.5130 - recall: 0.0460

KeyboardInterrupt: 

In [38]:
model_elu.evaluate(x_test, y_test)

313/313 [==============================] - 5s 15ms/step - loss: 2.2234 - accuracy: 0.5127 - precision: 0.5573 - recall: 0.4784


[2.2233879566192627,
 0.5127000212669373,
 0.5572510361671448,
 0.47840002179145813]

training accuracy: 82%
testing accuracy: 51.2%

### SELU

In [48]:
# Modify the layers to use ELU activation
LAYERS = [
    tf.keras.layers.Flatten(input_shape=[32, 32, 3], name="inputLayer"),
    tf.keras.layers.Dense(300, activation="selu", kernel_initializer = "lecun_normal" ,name="hiddenLayer1"),
    tf.keras.layers.Dense(200, activation="selu", kernel_initializer = "lecun_normal" , name="hiddenLayer2"),
    tf.keras.layers.Dense(100, activation="selu", kernel_initializer = "lecun_normal" , name="hiddenLayer3"),
    tf.keras.layers.Dense(10, activation="selu", kernel_initializer = "lecun_normal" , name="outputLayer")
]

In [49]:
model_selu = tf.keras.models.Sequential(LAYERS)

In [50]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model_selu.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [51]:
history_selu = model_selu.fit(x_train,y_train, 
              epochs=50,batch_size = 128,
              validation_data=(x_test, y_test))

Epoch 1/50


2023-05-12 15:08:04.244497: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: Graph execution error:

Detected at node 'assert_greater_equal/Assert/AssertGuard/Assert' defined at (most recent call last):
    File "/Users/pratikhotchandani/opt/anaconda3/envs/tensorflow/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/pratikhotchandani/opt/anaconda3/envs/tensorflow/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/pratikhotchandani/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/pratikhotchandani/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/pratikhotchandani/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/pratikh

### Batch normalization

In [59]:
model_bn = keras.models.Sequential([
    keras. layers.Flatten(input_shape= [32,32,3]),
    keras. layers. BatchNormalization(), 
    keras. layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(), 
    keras. layers.Activation("elu"),
    keras. layers. Dense(100, kernel_initializer="he_normal", use_bias=False), 
    keras. layers.Activation("elu"), 
    keras. layers.BatchNormalization(), 
    keras.layers.Dense(10, activation="softmax")
])

In [60]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model_bn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

In [61]:
history_bn = model_bn.fit(x_train,y_train, 
              epochs=50,batch_size = 128,
              validation_data=(x_test, y_test))

Epoch 1/50


2023-05-12 15:36:37.076398: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - ETA: 0s - loss: 1.6811 - accuracy: 0.4120 - precision: 0.6333 - recall: 0.1672

2023-05-12 15:36:47.366491: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 12s 28ms/step - loss: 1.6811 - accuracy: 0.4120 - precision: 0.6333 - recall: 0.1672 - val_loss: 1.5356 - val_accuracy: 0.4627 - val_precision: 0.6916 - val_recall: 0.2072
Epoch 2/50
391/391 [==============================] - 10s 24ms/step - loss: 1.4809 - accuracy: 0.4809 - precision: 0.6861 - recall: 0.2552 - val_loss: 1.4559 - val_accuracy: 0.4855 - val_precision: 0.6787 - val_recall: 0.2843
Epoch 3/50
391/391 [==============================] - 10s 25ms/step - loss: 1.3856 - accuracy: 0.5127 - precision: 0.7032 - recall: 0.3023 - val_loss: 1.3994 - val_accuracy: 0.5033 - val_precision: 0.6848 - val_recall: 0.3157
Epoch 4/50
391/391 [==============================] - 10s 25ms/step - loss: 1.3105 - accuracy: 0.5368 - precision: 0.7252 - recall: 0.3399 - val_loss: 1.3652 - val_accuracy: 0.5186 - val_precision: 0.6809 - val_recall: 0.3498
Epoch 5/50
391/391 [==============================] - 10s 25ms/step - loss: 1.2548 - accuracy: 0.5590 - pre

391/391 [==============================] - 11s 27ms/step - loss: 0.3538 - accuracy: 0.8768 - precision: 0.9036 - recall: 0.8500 - val_loss: 2.0760 - val_accuracy: 0.5185 - val_precision: 0.5501 - val_recall: 0.4938
Epoch 38/50
391/391 [==============================] - 10s 25ms/step - loss: 0.3461 - accuracy: 0.8777 - precision: 0.9041 - recall: 0.8538 - val_loss: 2.0990 - val_accuracy: 0.5250 - val_precision: 0.5539 - val_recall: 0.4998
Epoch 39/50
391/391 [==============================] - 11s 28ms/step - loss: 0.3313 - accuracy: 0.8853 - precision: 0.9080 - recall: 0.8609 - val_loss: 2.1279 - val_accuracy: 0.5208 - val_precision: 0.5507 - val_recall: 0.4954
Epoch 40/50
391/391 [==============================] - 10s 24ms/step - loss: 0.3132 - accuracy: 0.8902 - precision: 0.9114 - recall: 0.8693 - val_loss: 2.1906 - val_accuracy: 0.5195 - val_precision: 0.5465 - val_recall: 0.4968
Epoch 41/50
391/391 [==============================] - 10s 25ms/step - loss: 0.3117 - accuracy: 0.8907 -

In [62]:
model_bn.evaluate(x_test, y_test)

313/313 [==============================] - 5s 14ms/step - loss: 2.4785 - accuracy: 0.5233 - precision: 0.5419 - recall: 0.5066


[2.478476047515869, 0.523300051689148, 0.5418761372566223, 0.506600022315979]

Training accuracy: 91%
Testing accuracy: 52.3%